In [16]:
#Importing Libraries
import pandas as pd
import numpy as np

df = pd.read_csv("./data/Hotel_Reviews.csv")

df= df.drop(['Review_Date','Review_Total_Negative_Word_Counts','Review_Total_Positive_Word_Counts','days_since_review','Additional_Number_of_Scoring','Total_Number_of_Reviews_Reviewer_Has_Given','Positive_Review','Negative_Review','Reviewer_Nationality','Reviewer_Score'], axis=1)
uniqueHotelsDF = df.drop_duplicates(subset=['Hotel_Name'])
uniqueHotelsDF=uniqueHotelsDF.drop(['Tags','Hotel_Address'],axis=1)
uniqueHotelsDF['lat'].fillna(0, inplace=True)
uniqueHotelsDF['lng'].fillna(0, inplace=True)
#uniqueHotelsDF


In [17]:
hotelLeisureCounts = df[df['Tags'].str.contains('Leisure')].groupby('Hotel_Name')['Hotel_Name'].count().sort_values(ascending=False)
print "BusinessCounts: ", len(hotelLeisureCounts)
#print(hotelLeisureCounts)

hotelBusinessCounts = df[df['Tags'].str.contains('Business')].groupby('Hotel_Name')['Hotel_Name'].count().sort_values(ascending=False)
print "Leisure Counts: ",len(hotelBusinessCounts)
#print(hotelBusinessCounts)

hotelSoloCounts = df[df['Tags'].str.contains('Solo')].groupby('Hotel_Name')['Hotel_Name'].count().sort_values(ascending=False)
print "Solo Traveller Counts: ",len(hotelSoloCounts)

hotelCoupleCounts = df[df['Tags'].str.contains('Couple')].groupby('Hotel_Name')['Hotel_Name'].count().sort_values(ascending=False)
print "Solo Couple Counts: ",len(hotelCoupleCounts)

hotelGroupCounts = df[df['Tags'].str.contains('Group')].groupby('Hotel_Name')['Hotel_Name'].count().sort_values(ascending=False)
print "Solo Group Counts: ",len(hotelGroupCounts)

BusinessCounts:  1492
Leisure Counts:  1488
Solo Traveller Counts:  1491
Solo Couple Counts:  1492
Solo Group Counts:  1481


In [18]:
uniqueHotelsDF.loc[:,'Business']=0;
uniqueHotelsDF.loc[:,'Leisure']=0;
uniqueHotelsDF.loc[:,'Solo']=0;
uniqueHotelsDF.loc[:,'Couple']=0;
uniqueHotelsDF.loc[:,'Group']=0;
#uniqueHotelsDF

In [19]:
uniqueHotels=uniqueHotelsDF['Hotel_Name'].tolist()
for hotel in uniqueHotels:
    indx=uniqueHotelsDF.index[uniqueHotelsDF['Hotel_Name'] == hotel]
    #Update Trip type columns
    if hotel in hotelBusinessCounts and hotel in hotelLeisureCounts:
        if hotelLeisureCounts[hotel] > hotelBusinessCounts[hotel]:
            uniqueHotelsDF.loc[indx, 'Leisure'] = 1
        elif hotelLeisureCounts[hotel] < hotelBusinessCounts[hotel]:
            uniqueHotelsDF.loc[indx, 'Business'] = 1
        else:
            uniqueHotelsDF.loc[indx, 'Leisure'] = 1
            uniqueHotelsDF.loc[indx, 'Business'] = 1
    elif hotel in hotelBusinessCounts and hotel not in hotelLeisureCounts:
        uniqueHotelsDF.loc[indx, 'Business'] = 1
    elif hotel in hotelLeisureCounts and hotel not in hotelBusinessCounts:
        #print ("no business tag reviews for: ",hotel)
        uniqueHotelsDF.loc[indx, 'Leisure'] = 1
        
    #Update Traveller type columns
    #print hotelCoupleCounts['Best Western Amiral Hotel']
    if hotel in hotelSoloCounts and hotel in hotelCoupleCounts and hotel in hotelGroupCounts:
        soloCount=hotelSoloCounts[hotel]
        coupleCount=hotelCoupleCounts[hotel]
        groupCount=hotelGroupCounts[hotel]

        if soloCount>coupleCount and soloCount>groupCount:
            uniqueHotelsDF.loc[indx, 'Solo'] = 1
        elif coupleCount>soloCount and coupleCount>groupCount:
            uniqueHotelsDF.loc[indx, 'Couple'] = 1
        elif groupCount>soloCount and groupCount>coupleCount:
            uniqueHotelsDF.loc[indx, 'Group'] = 1
        elif soloCount==coupleCount and soloCount>groupCount:
            uniqueHotelsDF.loc[indx, 'Solo'] = 1
            uniqueHotelsDF.loc[indx, 'Couple'] = 1
        elif soloCount==groupCount and soloCount>coupleCount:
            uniqueHotelsDF.loc[indx, 'Solo'] = 1
            uniqueHotelsDF.loc[indx, 'Group'] = 1
        elif coupleCount==groupCount and coupleCount>soloCount:
            uniqueHotelsDF.loc[indx, 'Couple'] = 1
            uniqueHotelsDF.loc[indx, 'Group'] = 1
        else:
            uniqueHotelsDF.loc[indx, 'Solo'] = 1
            uniqueHotelsDF.loc[indx, 'Couple'] = 1
            uniqueHotelsDF.loc[indx, 'Group'] = 1
    
uniqueHotelsDF

,Average_Score,Hotel_Name,Total_Number_of_Reviews,lat,lng,Business,Leisure,Solo,Couple,Group
0,7.7,Hotel Arena,1403,52.360576,4.915968,0,1,0,1,0
405,8.5,K K Hotel George,1831,51.491888,-0.194971,0,1,0,1,0
971,9.2,Apex Temple Court Hotel,2619,51.513734,-0.108751,0,1,0,1,0
2008,7.7,The Park Grand London Paddington,4380,51.514218,-0.180903,0,1,0,1,0
3778,8.4,Monhotel Lounge SPA,171,48.874348,2.289733,0,1,0,1,0
3813,7.2,Kube Hotel Ice Bar,197,48.886570,2.358833,0,1,0,1,0
3832,8.0,The Principal London,3150,51.522622,-0.125160,0,1,0,1,0
5257,8.4,Park Plaza County Hall London,6117,51.501400,-0.116009,0,1,0,1,0
7480,9.2,One Aldwych,259,51.511783,-0.119417,0,1,0,1,0
7616,8.9,Splendid Etoile,486,48.874707,2.293676,0,1,0,1,0


In [20]:
uniqueHotelsDF=uniqueHotelsDF.reset_index()


In [21]:
processedHotelsDF=pd.DataFrame(uniqueHotelsDF)
processedHotelsDF = processedHotelsDF.drop(['index','Hotel_Name'],axis=1)
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
similarityDF=cosine_similarity(processedHotelsDF,processedHotelsDF)
similarityDF = pd.DataFrame(similarityDF)
#processedHotelsDF

In [22]:
titles = uniqueHotelsDF['Hotel_Name']
indices = pd.Series(uniqueHotelsDF.index, index=uniqueHotelsDF['Hotel_Name'])
#similarityDF

In [23]:
def get_recommendations(title):
    idx = indices[title]
    #print 'dix=',idx
    sim_scores = list(enumerate(similarityDF[idx]))
    #print 'sim_score',sim_scores.shape
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [24]:
get_recommendations('Milestone Hotel Kensington').head(10)

469                           Dukes Hotel
141       London Marriott Hotel Park Lane
226                      The Pillar Hotel
672    Mandarin Oriental Hyde Park London
152                            The Goring
302                  The Kings Head Hotel
89                        Marlin Waterloo
191                         Egerton House
34                        Hotel Le 10 BIS
555           H tel Barri re Le Fouquet s
Name: Hotel_Name, dtype: object

In [25]:
y = processedHotelsDF['Average_Score'].values
X = processedHotelsDF.drop('Average_Score',axis=1).values
#Appliyng XGBoost to our data
from sklearn.model_selection import train_test_split
import xgboost as xgb
X_train ,X_test ,y_train ,y_test = train_test_split(X,y ,test_size=0.2 ,random_state = 20)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)

In [14]:
param = {}
#Using gamma regression 
param['objective'] = 'reg:gamma'
param['eta'] = 0.2
param['max_depth'] = 4
param['silent'] = 1
param['nthread'] = 4
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 25

bst = xgb.train(param, xg_train, num_round, watchlist)
pred = bst.predict(xg_test)

[0]	train-gamma-nloglik:13.504	test-gamma-nloglik:13.608
[1]	train-gamma-nloglik:11.3154	test-gamma-nloglik:11.4017
[2]	train-gamma-nloglik:9.55447	test-gamma-nloglik:9.62638
[3]	train-gamma-nloglik:8.14272	test-gamma-nloglik:8.20283
[4]	train-gamma-nloglik:7.01566	test-gamma-nloglik:7.06611
[5]	train-gamma-nloglik:6.12034	test-gamma-nloglik:6.16287
[6]	train-gamma-nloglik:5.41321	test-gamma-nloglik:5.44927
[7]	train-gamma-nloglik:4.85844	test-gamma-nloglik:4.8892
[8]	train-gamma-nloglik:4.42653	test-gamma-nloglik:4.45296
[9]	train-gamma-nloglik:4.09317	test-gamma-nloglik:4.11606
[10]	train-gamma-nloglik:3.83835	test-gamma-nloglik:3.85835
[11]	train-gamma-nloglik:3.64564	test-gamma-nloglik:3.66328
[12]	train-gamma-nloglik:3.50156	test-gamma-nloglik:3.51727
[13]	train-gamma-nloglik:3.39515	test-gamma-nloglik:3.40929
[14]	train-gamma-nloglik:3.31755	test-gamma-nloglik:3.33041
[15]	train-gamma-nloglik:3.26168	test-gamma-nloglik:3.2735
[16]	train-gamma-nloglik:3.22197	test-gamma-nloglik:3.

In [15]:
#Calculating Error
error = pred - y_test
error = abs(error)
error_rate = np.mean(error)/np.mean(y_test)
print("error rate = %f " %error_rate )

error rate = 0.090558 
